<a href="https://colab.research.google.com/github/miam-bonbon/assignment-adv-nlp/blob/main/adv_nlp_assignment_mh_03_baseline_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Base Classifier

by Michael Hunziker

## Summary
The aim of this notebook is to train a *Base Classifier* using TF-IDF and a standard ML classifier with default hyperparameters.

The aim of this notebook is to train a *Base Classifier* using TF-IDF and a standard ML classifier with default hyperparameters. This builds a *base line* upon which more advanced models (i.e. deep learning ones) should show better performance in order to be considered. Additionally, it also investigate the how much individual features (i.e. words) contribute to the class assignment (feature importance) in order to get an idea if the classifier makes *reasonable* decisions.

In a second step, this notebook uses hyperparameter tuning to automatically improve the performance of the Base Classifier.


*   Prepare Dataset




<a href="https://git.com">
  <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Let's do some imports

In [ ]:
%%capture

!pip install 'fhnw-nlp-utils>=0.9.8,<0.10.0'

from fhnw.nlp.utils.storage import download
from fhnw.nlp.utils.storage import load_dataframe

import pandas as pd
import numpy as np

from fhnw.nlp.utils.system import set_log_level
from fhnw.nlp.utils.system import system_info

set_log_level()
print(system_info())

In [ ]:
%%time

!rm "./data/COVIDSenti_cleaned.parq"

download("https://github.com/miam-bonbon/nlp-data/raw/refs/heads/main/data/COVIDSenti_cleaned.parq", "data/COVIDSenti_cleaned.parq")
data = load_dataframe("data/COVIDSenti_cleaned.parq")
data.shape

CPU times: user 221 ms, sys: 93.6 ms, total: 315 ms
Wall time: 369 ms


(89190, 4)

In [ ]:
data.head(3)

,tweet,label,cleaned_tweet,lang
0,Coronavirus | Human Coronavirus Types | CDC ht...,neu,Coronavirus | Human Coronavirus Types | CDC ht...,en
1,"@shehryar_taseer That‚Äôs üíØ true , \nCorona...",neu,"Thats true , Corona virus swine flue Bird...",en
2,"TLDR: Not SARS, possibly new coronavirus. Diff...",neg,"TLDR: Not SARS, possibly new coronavirus. Diff...",en


Remove neutral sentiments, since we classify binary

In [ ]:
# remove all neutral sentimens
data = data.loc[(data["label"] != "neu")]
# remove all nan
data = data.dropna()

data.shape

(22516, 4)

In [ ]:
data.head(3)

,tweet,label,cleaned_tweet,lang
2,"TLDR: Not SARS, possibly new coronavirus. Diff...",neg,"TLDR: Not SARS, possibly new coronavirus. Diff...",en
8,@tezuma75 Why #CCP keep on saying unknown caus...,neg,Why #CCP keep on saying unknown cause of pneu...,en
11,I always feel weird hoping for another coronav...,neg,I always feel weird hoping for another coronav...,en


In [ ]:
val_counts = data["label"].value_counts().sort_index()
val_counts

,count
label,
neg,16273
pos,6243
